<a href="https://colab.research.google.com/github/Jarcos09/Tareas/blob/main/docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docling
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 40.8 MB/s eta 0:00:00
   ━━

In [ ]:
from docling.document_converter import DocumentConverter

import pandas as pd
import os
from openpyxl import load_workbook

In [ ]:
# --- Configuración ---
# Puedes usar una URL o una ruta a un archivo PDF local.
# Para este ejemplo, usaré un PDF de ejemplo que podrías reemplazar.
# Si tienes un PDF local, asegúrate de que la ruta sea correcta.

# pdf_source = "ruta/a/tu/documento.pdf" # Descomenta y usa tu propio PDF local
#pdf_source = os.path.join('Documentos', 'OF.pdf')
pdf_source = "/content/OF.pdf"
# --- Inicializar el conversor de Docling ---
converter = DocumentConverter()

print(f"Procesando el PDF desde: {pdf_source}")

Procesando el PDF desde: /content/OF.pdf


In [ ]:
def imprime_tabla(tabla_data, num_rows, num_cols):

  file_name = "tablas_of.xlsx"

  table_matrix = [[None for _ in range(num_cols)] for _ in range(num_rows)]

  # 3. Llenar la matriz con el texto de las celdas
  for cell in tabla_data.table_cells:
      row_start = cell.start_row_offset_idx
      col_start = cell.start_col_offset_idx
      row_span = cell.row_span
      col_span = cell.col_span
      text = cell.text

      # Asignar el texto a la posición correcta, considerando row_span y col_span
      # Aunque en tus datos todos son span 1, este código es más robusto.
      for r_offset in range(row_span):
          for c_offset in range(col_span):
              current_row = row_start + r_offset
              current_col = col_start + c_offset
              if 0 <= current_row < num_rows and 0 <= current_col < num_cols:
                  table_matrix[current_row][current_col] = text
              else:
                  print(f"Advertencia: Celda fuera de límites calculados: {cell}")

  # 4. Identificar los nombres de las columnas (headers)
  column_names = [None] * num_cols
  for cell in tabla_data.table_cells:
      if cell.column_header:
          # Asume que los encabezados están en el primer row_offset_idx de los headers
          if cell.start_row_offset_idx == 0:
              col_idx = cell.start_col_offset_idx
              if 0 <= col_idx < num_cols:
                  column_names[col_idx] = cell.text

  # Si algunos nombres de columna no se encontraron o son None (por ejemplo, si la primera fila no tuviera encabezados para todas las columnas), puedes usar nombres genéricos
  for i, name in enumerate(column_names):
      if name is None:
          column_names[i] = f'Columna_{i}'

  # 5. Crear el DataFrame de pandas
  # Eliminar la fila de encabezados de la matriz si los encabezados se usarán como nombres de columna
  # En tu caso, los encabezados están en la fila 0 (start_row_offset_idx: 0)
  # Así que los datos "reales" de la tabla empiezan desde la fila 1.
  data_rows = table_matrix[1:] # Excluye la primera fila que son los headers

  df = pd.DataFrame(data_rows, columns=column_names)

  # Opcional: Si la primera columna 'No' es un índice y no un dato, puedes establecerla como índice
  # Si es un dato que quieres conservar, simplemente omite esta línea.
  if 'Columna_0' in df.columns:
      df = df.set_index('Columna_0')

  # Mostrar el DataFrame
  print("\nDataFrame generado:")
  print(df)


  sheet_name = f"Hoja_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"

  if os.path.exists(file_name):
    # Cargar el libro existente
    book = load_workbook(file_name)
    with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df.to_excel(writer, sheet_name=sheet_name)
  else:
    # Crear un nuevo archivo Excel
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=sheet_name)


  print(f"DataFrame guardado en la hoja '{sheet_name}' del archivo '{file_name}'.")

In [ ]:
try:
    result = converter.convert(pdf_source)
    doc = result.document

    print("\n--- Información General del Documento ---")
    dict_repr = doc.export_to_dict()
    num_texts = len(dict_repr.get('texts', []))
    num_tables = len(dict_repr.get('tables', []))

    print(f"Elementos de texto encontrados: {num_texts}")
    print(f"Tablas encontradas: {num_tables}")

except Exception as e:
    print(f"Ocurrió un error al procesar el documento: {e}")


--- Información General del Documento ---
Elementos de texto encontrados: 81
Tablas encontradas: 5


In [ ]:
 # --- Extraer y mostrar los primeros párrafos ---
print("\n--- Párrafos ---")
# Itera sobre los elementos del documento para encontrar párrafos.
# Docling organiza el contenido en una estructura jerárquica.
for item, _ in doc.iterate_items():
  if item.label == "text":
    palabras = item.text.split()
    lineas = [' '.join(palabras[i:i+10]) for i in range(0, len(palabras), 10)]
    parrafo_formateado = '\n'.join(lineas)
    print(f"• {parrafo_formateado}\n")





--- Párrafos ---
• Subdelegación Tepic

• REMITENTE SUBDELEGACIÓN TEPIC

• C. del Área de Correspondencia OOAD Nayarit Jefe

• FECHA 02/09/2025

• Presente

• Clave Presupuestal:

• Me permito adjuntarle sobre(s).

• Para ser enviado(s) al (a los) siguiente(s) destinatario(s):

• X

• Este servicio se recibe hasta las 12:00 pm para su
envío.

• Valija delegacional

• Requiriéndose el siguiente servicio:

• Correo ordinario

• 04'28

• Correo Registrado (RASTREO)

• Este servicio se recibe hasta las 10:00 am para su
envío.

• Por la oficina remitente

• 0 2 SET. 2025

• RUBFELIZABETH DAVALOS REYES Nombre) firma

• (Sello de recepción)

• Esta forma deberá Ilenarse en dos tantos. Original el Área
de Correspondencia. Copia para el Remitente para

• nv

• INSTITUTO MEXICANO DEL SEGURO SOCIAL Órgano de Operación Administrativa Desconcentrada
en Nayarit Subdelegación Tepic Departamento de Afiliación Vigencia

• Mtra. Adriana Alejandra Pedroza Márquez Titular de la Coordinación de
Afiliación

In [ ]:
print("\n--- Tablas ---")

for item, _ in doc.iterate_items():
    if item.label == "table":

        max_row_idx = 0
        max_col_idx = 0

        for cell in item.data.table_cells:
            if cell.end_row_offset_idx > max_row_idx:
                max_row_idx = cell.end_row_offset_idx
            if cell.end_col_offset_idx > max_col_idx:
                max_col_idx = cell.end_col_offset_idx

        num_rows = max_row_idx
        num_cols = max_col_idx

        print(f"\n• La tabla tiene {num_rows} renglones y {num_cols} columnas.")
        imprime_tabla(item.data, num_rows, num_cols)


--- Tablas ---

• La tabla tiene 4 renglones y 7 columnas.

DataFrame generado:
  Progre sivo              Referencia  \
0        None                    None   
1        None  1991019001 10/055/2025   
2       TOTAL                    None   

                                             Destino Documentos que se envía  \
0                                               None               Origi nal   
1  MTRA. ADRIANA ALEJANDRA PEDROZA MARQUEZ TITULA...                       X   
2                                               None                    None   

  Columna_4   Columna_5     Observacior Asunto  
0     Copia  Con Anexos                   es 0  
1      None        None  PATRONES CON CAUSA 50  
2      None        None                   None  
DataFrame guardado en la hoja 'Hoja_20250903_171208' del archivo 'tablas_of.xlsx'.

• La tabla tiene 8 renglones y 2 columnas.

DataFrame generado:
  Registro Patronal                  Nombre del patrón
0       D3553164103        CARMEN 